In [1]:
import requests
import time
import csv
import pandas as pd
import os
from web3 import Web3
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv('INFURA_API_KEY')

infura_mainnet_url = f'https://mainnet.infura.io/v3/{api_key}'
infura_polygon_url = f'https://polygon-mainnet.infura.io/v3/{api_key}'
infura_base_url = f'https://base-mainnet.infura.io/v3/{api_key}'
infura_optimism_url = f'https://optimism-mainnet.infura.io/v3/{api_key}'
infura_arbitrum_url = f'https://arbitrum-mainnet.infura.io/v3/{api_key}'

web3 = Web3(Web3.HTTPProvider(infura_polygon_url))

def is_contract(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return len(code) > 2

In [2]:
all_addresses = set()

df = pd.read_csv('../data/1_year/combined_transfers_1_year.csv')

all_addresses.update(df['from'].unique())
all_addresses.update(df['to'].unique())

distinct_addresses_df = pd.DataFrame(list(all_addresses), columns=['address'])

distinct_addresses_df.to_csv('../data/1_year/distinct_addresses_1_year.csv', index=False)

In [16]:
df1 = pd.read_csv('../data/1_year/distinct_addresses_1_year.csv')

df2 = pd.read_csv('../data/node_table.csv')

filtered_df = df1[~df1['address'].isin(df2['address'])]

filtered_df.to_csv('../data/1_year/addresses_to_check_1_year.csv', index=False)

In [3]:
addresses_df = pd.read_csv('../data/polygon/distinct_addresses_1_year.csv')

In [12]:
addresses_slice = addresses_df.iloc[10844:52615]

In [ ]:
for i, row in addresses_slice.iterrows():
    print(i)
    address = row['address']
    type = is_contract(address)
    addresses_slice.at[i, 'type'] = type
    time.sleep(1/25)

In [16]:
# addresses_slice['type'] = addresses_slice['address'].apply(is_contract)
addresses_slice.to_csv('../data/polygon/checked_addresses_1_year_1.csv', index=False)